<p style="font-size:35px; text-align:center; font-weight:bold">CMAAS run SOM clustering method</p>
<p style="font-size:17px; text-align:left">Ina Storch 06-11-2023 </p>
<p style="font-size:17px; text-align:left">Note: This notebook is designed to run SOM using preprocessed data gained from the datacube from: Lawley et al., 2021. </p>
<p style="font-size:17px; text-align:left">Reference: Lawley, C.J.M., McCafferty, A.E., Graham, G.E., Gadd, M.G., Huston, D.L., Kelley, K.D., Paradis, S., Peter, J.M., and Czarnota, K., 2021. Datasets to support prospectivity modelling for sediment-hosted Zn-Pb mineral systems; Geological Survey of Canada, Open File 8836, 1 .zip file. https://doi.org/10.4095/329203</p>

<p style="font-size:19px; text-align:left; font-weight:bold">1) Import libraries, define SOM arguments</p>

In [1]:
from beak.methods.som.nextsomcore.nextsomcore import NxtSomCore
import pickle

import beak.methods.som.argsSOM as asom

args = asom.Args()

#base_path = "d:/Programmieren/00_Repositories/CMAAS"
base_path = "d:/projekte/cmaas/repository"

<p style="font-size:19px; text-align:left; font-weight:bold">2) Provide input file(s). 

Input data can eighter be in LRN file format (a table with X, Y, Z and one line with the values for each data layer) or geoTIF file format. Choose one.

When you run a Docker container, you need to specify which directories or paths from your local machine should be mounted into the container since Docker containers are isolated environments, and by default, they can only access files within their own filesystem or volumes.

Alternatively, copy your input data to beak-ta3\src\beak\data. In case of GeoTIF input files, you can create subfolders and collect all TIF files that you want to use for SOM in there. In the following code cell you can provide a file path and a patterns (like "*.tif") for files within that specific folder that you want to use for SOM.

If input file format is LRN, skip the next code cell and provide the file name (including file path) with args.input_file. LRN files should not contain nan values since so far noData handeling is implemented for GeoTIF input only. 

<p style="font-size:19px; text-align:left; font-weight:bold">Either: Data input in geoTiff format:

List geoTiff files, separated by komma: ["path/name1.tiff","path/name2.tiff"] </br>

Here an option is provided how to compose such a list: provide a file path and one or more file patterns (or a list of file names) to select all files matching this pattern inside the specified folder. The file patterns can also include subfolder specifications and * to select all files within this subfolder ('subfolder/*.tif') .

In [3]:
#--------------- Option 1: specify data path and file patterns 
#import sys
#if sys.version_info < (3, 9):
#    from importlib_resources import files
#else:
#    from importlib.resources import files

#BASE_PATH = (files("beak.data") / "LAWLEY22-EXPORT" / "EPSG_3857_RES_5000" / "CLIPPED_USC")

#PATH_NUMERICAL = BASE_PATH / "NUMERICAL_SCALED_MINMAX" / "*.tif"
#PATH_CATEGORICAL = BASE_PATH / "CATEGORICAL" / "*" /"*.tif"

#PATH_NUMERICAL = BASE_PATH / "NUMERICAL_SCALED_MINMAX" / "Magnetic_LongWavelength_HGM.tif"
#PATH_CATEGORICAL = BASE_PATH / "CATEGORICAL" / "Geology_Lithology_Majority_Igneous_Intrusive_Felsic.tif"

#file_patterns = [str(PATH_NUMERICAL), str(PATH_CATEGORICAL)]


#args.input_file = args.create_list_from_pattern("", file_patterns)

#--------------- Option 2: load a list of GeoTIF files from TXT file 
# Specify the file path
file_path = base_path + "/beak-ta3/experiments/mvt_nat/som/results/input_file_list.txt"
# optional:
#file_path_label = base_path + "/beak-ta3/experiments/mvt_nat/som/results/label_file_list.txt"


args.input_file = args.create_list_from_file(file_path)

#---------------
args.geotiff_input=args.input_file      # geotiff_input files, separated by komma, to write GeoTIF out (only first line is used to get the geotransform and projection information to set output GeoFIT geotransform and projection)

Number of files added: 2
Files:
d:\projekte\cmaas\repository\beak-ta3\src\beak\data\LAWLEY22-EXPORT\EPSG_3857_RES_5000\CLIPPED_USC\CATEGORICAL\Geology_Dictionary_Felsic\Geology_Dictionary_Felsic_Present.tif
d:\projekte\cmaas\repository\beak-ta3\src\beak\data\LAWLEY22-EXPORT\EPSG_3857_RES_5000\CLIPPED_USC\NUMERICAL_IMPUTED_SCALED_STANDARD\Magnetic_HGM.tif


<p style="font-size:19px; text-align:left; font-weight:bold">Or: Data input in LRN format (data table):

Specify file path and name!

In [4]:
#args.input_file="/methods/methods/som/data/input/SOM_grav_mag.lrn"

<p style="font-size:19px; text-align:left; font-weight:bold">3) Specify parameter for SOM. 

In [5]:
     
args.output_folder = base_path + "/beak-ta3/experiments/mvt_nat/som/results" # Folder to save som dictionary and cluster dictionary
args.output_file_somspace = args.output_folder+"/result_som.txt"   # DO NOT CHANGE! Text file that will contain calculated values: som_x som_y b_data1 b_data2 b_dataN umatrix cluster in geospace.

args.som_x = 20                # X dimension of generated SOM
args.som_y = 20               # Y dimension of generated SOM
args.epochs = 10               # Number of epochs to run

# Additional optional parameters:
args.outgeofile= args.output_folder+"/result_geo.txt"             # DO NOT CHANGE!
args.output_file_geospace=args.outgeofile   # Text file that will contain calculated values: {X Y Z} data1 data2 dataN som_x som_y cluster b_data1 b_data2 b_dataN in geospace.

args.kmeans=True          # Run k-means clustering (true, false)
args.kmeans_init = 5           # Number of initializations
args.kmeans_min = 29            # Minimum number of k-mean clusters
args.kmeans_max = 30           # Maximum number of k-mean clusters

args.neighborhood = 'gaussian'     # Shape of the neighborhood function. gaussian or bubble
args.std_coeff = 0.5               # Coefficient in the Gaussian neighborhood function
args.maptype = 'toroid'            # Type of SOM (sheet, toroid)
args.initialcodebook = None        # File path of initial codebook, 2D numpy.array of float32.
args.radius0 = 0                   # Initial size of the neighborhood
args.radiusN = 1                   # Final size of the neighborhood
args.radiuscooling = 'linear'      # Function that defines the decrease in the neighborhood size as the training proceeds (linear, exponential)
args.scalecooling = 'linear'       # Function that defines the decrease in the learning scale as the training proceeds (linear, exponential)
args.scale0 = 0.1                  # Initial learning rate
args.scaleN = 0.01                 # Final learning rate
args.initialization = 'random'     # Type of SOM initialization (random, pca)
args.gridtype = 'rectangular'      # Type of SOM grid (hexagonal, rectangular)

args.label = True                  # Whether data contains label column, True or False
args.label_geotiff_file = args.create_list_from_file(args.output_folder + "/label_file_list.txt")      # geotiff_input file (None)


Number of files added: 1
Files:
d:\projekte\cmaas\repository\beak-ta3\src\beak\data\LAWLEY22-EXPORT\EPSG_3857_RES_5000\CLIPPED_USC\GROUND_TRUTH\POLYGONS\Training_MVT_Present.tif


<p style="font-size:19px; text-align:left; font-weight:bold">4) Run SOM 

Before running SOM - clean up existing files and move them to a subfolder.

Run SOM with parameters specified above and save the results. Uses NxtSomCore package to do the actual work. 

In [6]:
import beak.methods.som.do_nextsomcore_save_results as dnsr
import beak.methods.som.move_to_subfolder as mts
import warnings

# move or remove existing SOM output files from previous runs into subfolder
#mts.move_som_results(args.output_folder, "old_results")
mts.remove_som_results(args.output_folder)

# run SOM
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    dnsr.run_SOM(args)
    

Load data
    Execution time: 0.09685444831848145 seconds
Run SOM
    Execution time: 8.47169828414917 seconds
Clustering progress:
0.00%
10.00%
20.00%
30.00%
40.00%
50.00%
60.00%
70.00%
80.00%
90.00%
100% Clustering completed.
    Execution time: 0.4708526134490967 seconds
Save geo space results
         combine data colums for output geo file (for large data arrays memory usage might be a concern)
         savetxt
    Execution time: 8.228259563446045 seconds
Save SOM space results
    Execution time: 0.29526758193969727 seconds
Save SOM object to file
    Execution time: 0.003081798553466797 seconds
Write GeoTIFF file
     read_csv som_data
     read_csv geo_data
     Iterate over each geoTIF file:
          cluster
          b_Geology_Dictionary_Felsic_Present
          b_Magnetic_HGM
    Execution time: 1.842698097229004 seconds
Count cluster hit count
        Cluster hit count: 29 - 14289
        Cluster hit count: 28 - 93853
        Cluster hit count: 27 - 15831
        Cluster 

In [7]:
import beak.methods.som.plot_som_results as plot
from IPython.display import Image, display, clear_output

# Load cluster dictionary
loaded_cluster_list = plot.load_cluster_dictionary(args.output_folder)
# Plot and save the Davies-Bouldin Index vs Number of Clusters
plot.plot_davies_bouldin(loaded_cluster_list, args.output_folder)

<p style="font-size:19px; text-align:left; font-weight:bold">5) Plot results.

Specify the parameters to plot the results and create figures. The Python script "plot_som_results.py" creates .png files of the results in som space, geospace and also creates boxplots.

Move figures into a sub folder. If the destination folder does not exist, it is created here. All file names are stored in a list that is used in the next step to show all output figures.

In [1]:
#import beak.methods.som.argsSOM as asom
#
#args = asom.Args()
#
##base_path = "d:/Programmieren/00_Repositories/CMAAS"
#base_path = "d:/projekte/cmaas/repository"
#args.output_folder = base_path + "/beak-ta3/experiments/mvt_nat/som/results"
#file_path = base_path + "/beak-ta3/experiments/mvt_nat/som/results/input_file_list.txt"
#args.input_file = args.create_list_from_file(file_path)
#args.output_file_somspace = args.output_folder+"/result_som.txt"
#args.som_x = 20
#args.som_y = 20
#args.output_file_geospace = args.output_folder+"/result_geo.txt"

Number of files added: 2
Files:
d:\projekte\cmaas\repository\beak-ta3\src\beak\data\LAWLEY22-EXPORT\EPSG_3857_RES_5000\CLIPPED_USC\CATEGORICAL\Geology_Dictionary_Felsic\Geology_Dictionary_Felsic_Present.tif
d:\projekte\cmaas\repository\beak-ta3\src\beak\data\LAWLEY22-EXPORT\EPSG_3857_RES_5000\CLIPPED_USC\NUMERICAL_IMPUTED_SCALED_STANDARD\Magnetic_HGM.tif


In [2]:
import beak.methods.som.argsPlot
import beak.methods.som.plot_som_results as plot
import beak.methods.som.move_to_subfolder as mts

argsP = beak.methods.som.argsPlot.Args()

argsP.outsomfile = args.output_file_somspace   # som calculation somspace output text file
argsP.som_x = args.som_x         # som x dimension
argsP.som_y = args.som_y         # som y dimension
argsP.input_file = args.input_file   # Input file
argsP.dir = args.output_folder            # directory where som.dictionary was safed to (/output/som.dictionary)
argsP.grid_type = 'rectangular' # grid type (square or hexa), (rectangular or hexagonal)
argsP.redraw = True       # whether to draw all plots, or only those required for clustering (true: draw all. false:draw only for clustering).
argsP.outgeofile = args.output_file_geospace     # som geospace results txt file
argsP.dataType = 'grid'       # Data type (scatter or grid)
argsP.noDataValue = '-9999'    # noData value

plot.run_plotting_script(argsP)

subfolder_name = "plots"
images, labels = mts.move_figures(args.output_folder, subfolder_name)

Setup figures
        Read som data execution time: 0.007882833480834961 seconds
        Read geo data execution time: 0.7650830745697021 seconds
    Execution time: 5.974203109741211 seconds
Plot geo space results


d:\projekte\cmaas\repository\beak-ta3\src\beak\methods\som\plot_som_results.py:398: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  z[z == noDataValue] = np.nan


    geospace plot no. 4 from 4
    q-error plot
    Execution time: 6.444802522659302 seconds
Plot Cluster result SOM space
    Plot Davies Bouldin index
    Plot cluster hit count
    Execution time: 0.9161975383758545 seconds
Plot SOM space results
    somspace plot no. 2 from 2
    Execution time: 1.9666166305541992 seconds
Plot Boxplots
    boxplot no. 2 from 2
    Execution time: 1.0786287784576416 seconds


Show all figures that were produced by "plot_som_results.py" and moved into a subfolder.

Boxplot:
> x axis is Cluster ID </br>
> Z axis is original data value </br>
> legend: number of data points in each cluster </br>

In [9]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
import ipyplot

# Clear Matplotlib cache
plt.close('all')

# Clear output
clear_output(wait=True)

tabs = [image.split('_')[-2] for image in labels]

print("List of figures:")
print(labels)
#print(tabs)

# Plot the images
#ipyplot.plot_images(images, max_images=50, img_width=250)
ipyplot.plot_class_tabs(images, tabs, max_imgs_per_tab=50, img_width=400)
ipyplot.plot_class_representations(images,  labels, img_width=200, show_url=False)


List of figures:
['geoplot_1.png', 'geoplot_2.png', 'geoplot_3.png', 'geoplot_4.png', 'somplot_1.png', 'somplot_2.png', 'somplot_3.png', 'somplot_4.png', 'somplot_5.png', 'somplot_6.png', 'boxplot_1.png', 'boxplot_2.png', 'db_score.png', 'cluster_hit_count.png']
